In [1]:
import pandas as pd
df = "C:/Users/THE KEY/Desktop/python_datum/pandas/data/weather/weather_beijing.xlsx"
wea = pd.read_excel(df)
wea = pd.DataFrame(wea)
wea.head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
0,2011-01-01 周六,-2°,-7,多云~阴,无持续风向微风,NaN
1,2011-01-02 周日,-2°,-7°,多云,无持续风向微风,NaN
2,2011-01-03 周一,-2°,-6°,多云~阴,西北风~北风3-4级~4-5级,NaN
3,2011-01-04 周二,-2°,-9°,晴,北风5-6级,NaN
4,2011-01-05 周三,-2°,-10°,晴,北风~无持续风向3-4级~微风,NaN


In [2]:
wea.loc[:,'最高温'] = wea['最高温'].str.replace("°","").astype(int)
wea.loc[:,'最低温'] = wea['最低温'].str.strip("°")

In [11]:
# 找出特殊群体nan
i = 0
for x in wea['最低温']:
    i += 1
    if x == 'nan':
        print(x)
        print(i)
#是 0 ，8 ，100，它们是nan，已改正

# 改正方法
wea.loc[[0],"最低温"] = '-7'
wea.loc[[8],"最低温"] = '-9'
wea.loc[[100],"最低温"] = '9'

wea['最低温'][455]
# 改正方法
wea.loc[[455],"最低温"] = '0'

In [13]:
# 这样会不会可以了呢？ 可以！！！！那之前那几个代码文件的for循环可以不用执行了
wea.loc[:,'最低温'] = wea['最低温'].astype(int)

### --------------------------------------------------------------

### 复现问题

In [18]:
# 只选出18年3月份的分析
condition = wea['日期'].str.startswith('2018-03')

In [29]:
wea[condition]["温差"] = wea["最高温"] - wea['最低温']

C:\Users\THE KEY\AppData\Local\Temp\ipykernel_8692\3566087602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wea[condition]["温差"] = wea["最高温"] - wea['最低温']


In [30]:
wea[condition].head()

,日期,最高温,最低温,天气,风力风向,空气质量指数
2604,2018-03-01 周四,8,-3,多云,西南风1-2级,46 优
2605,2018-03-02 周五,9,-1,晴~多云,北风1-2级,95 良
2606,2018-03-03 周六,13,3,多云~阴,北风1-2级,214 重度
2607,2018-03-04 周日,7,-2,阴~多云,东南风1-2级,144 轻度
2608,2018-03-05 周一,8,-3,晴,南风1-2级,94 良


### ----------------------------------------------------------

#####  原因是pandas不知道是返回view还是返回copy，所以这个链式操作有时会成功，有时会失败,因为上面代码是两步操作get+set

### ------------------------------------------------------------------

In [31]:
# 解决方式：改成一步操作
wea.loc[condition,'温差'] = wea["最高温"] - wea['最低温']

In [33]:
wea[condition].head()

,日期,最高温,最低温,天气,风力风向,空气质量指数,温差
2604,2018-03-01 周四,8,-3,多云,西南风1-2级,46 优,11
2605,2018-03-02 周五,9,-1,晴~多云,北风1-2级,95 良,10
2606,2018-03-03 周六,13,3,多云~阴,北风1-2级,214 重度,10
2607,2018-03-04 周日,7,-2,阴~多云,东南风1-2级,144 轻度,9
2608,2018-03-05 周一,8,-3,晴,南风1-2级,94 良,11


#### 总结
pandas不允许先筛选子dataframe，再进行写入修改

要么使用.loc实现一个步骤直接修改源dataframe

要么先复制一个子dataframe再一个步骤执行修改（.copy()）